In [1]:
import gesture_model
import torch
import image_dataset_loader
from torchvision.transforms import ToTensor

In [2]:
EPOCHS = 5

In [3]:
train_dataset, val_dataset, test_dataset = image_dataset_loader.create_datasets('new_train_data/data/')

discarding new_train_data/data/7 8.jpg
discarding new_train_data/data/8 12.jpg


the old version processed the images every epoch causing one epoch to be:
    trained in X seconds
training on all epochs yielded an accuracy of X2


after changing it so the training dataset has pre-processed, flattened landmarks, one epoch only:
    takes Y seconds
and yielded an accuracy of Y2

this is a Z% improvement

In [4]:
for ind, info in enumerate(val_dataset):
    path, lab = info

    flattened = image_dataset_loader.process_image(path, str(lab), True)

In [7]:
model = gesture_model.GestureModel(14)
model.to('cuda')
x = torch.randn((2, 126), dtype=torch.float64)
output = model(x)
assert output.shape == torch.Size((2, 14)), 'the output shape is wrong ' + str(output.shape)

print(f'In {x.shape} Out {output.shape}')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
import numpy as np
a = np.asarray([[1, 2], [3, 4]])
torch.from_numpy(a).dtype

torch.int64

In [ ]:
def train_model_one_epoch():
    epoch_loss = 0.
    debug_print_loss = 0.

    for i, data in enumerate(train_dataset):
        img_path, label = data

        optimizer.zero_grad()
    
        flattened = torch.from_numpy(image_dataset_loader.process_image(img_path, label, True))
        print(flattened.dtype)
        forward = model(flattened)

        cur_loss = loss(forward, label)
        cur_loss.backward()

        epoch_loss += cur_loss.item()
        if i % 5 == 0:
            debug_print_loss = epoch_loss / 32
            epoch_loss = 0
            print('  batch {} loss {}'.format(i + 1, debug_print_loss))
    return debug_print_loss

In [ ]:
model.linear1.weight.dtype

torch.float32

In [21]:
cur_epoch = 0
best_val_loss = 1_000_000

for epoch in range(EPOCHS):
    print(f'EPOCH {cur_epoch + 1}')

    model.train(True)
    epoch_loss = train_model_one_epoch()

    running_val_loss = 0.0
    model.eval()

    with torch.no_grad():
        for i, vdata in enumerate(val_dataset):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss(voutputs, vlabels)
            running_val_loss += vloss
    
    avg_vloss = running_val_loss / (i + 1)
    print(f'(LOSS) EPOCH {epoch}: train {epoch_loss} val {avg_vloss}')

    if avg_vloss < best_val_loss:
        best_val_loss = avg_vloss
        model_path = f'models/model_1_{epoch}'
    cur_epoch += 1

EPOCH 1
torch.float64


RuntimeError: mat1 and mat2 must have the same dtype